In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_unixtime, col, datediff, avg, current_date
import os

In [2]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.hadoop:hadoop-aws:3.3.1,io.delta:delta-spark_2.12:3.0.0 --conf "spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension" --conf "spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog" pyspark-shell'

In [3]:
spark = SparkSession.builder \
    .appName("Data_Analysis") \
    .config("spark.hadoop.fs.s3a.access.key", "ueVk36Brq2ZX8gCGGj1w") \
    .config("spark.hadoop.fs.s3a.secret.key", "12345678") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://172.29.8.228:9000") \
    .getOrCreate()

In [4]:
minio_bucket_name = "bucketd2d"
minio_path = "transient/historico"

In [5]:
delta_path = f"s3a://{minio_bucket_name}/{minio_path}"

In [6]:
merged_df = spark.read.format("delta").load(delta_path)

In [7]:
merged_df = merged_df.withColumn("data_hora", from_unixtime("dt"))

In [8]:
merged_df = merged_df.withColumn("dias_desde_hoje", datediff(current_date(), col("data_hora")))

In [9]:
filtered_df = merged_df.filter((col("dias_desde_hoje") >= 0) & (col("dias_desde_hoje") <= 30))

In [10]:
result = filtered_df.groupBy("city").agg(avg("tempe_max").alias("media_tempe_max"), avg("tempe_min").alias("media_tempe_min"))

In [11]:
result.show()

+---------+---------------+---------------+
|     city|media_tempe_max|media_tempe_min|
+---------+---------------+---------------+
|Itabaiana|         27.058|         27.058|
|  Lagarto|         27.442|         27.442|
|  Aracaju|          27.97|          27.97|
+---------+---------------+---------------+



In [12]:
spark.stop()